In [1]:
import os

# Check if we're in the root directory of rl-llm repo
if not os.path.basename(os.getcwd()) == 'rl-llm':
    # If we're in a subdirectory of rl-llm, find the root and cd to it
    current_path = os.getcwd()
    while os.path.basename(current_path) != 'rl-llm' and os.path.dirname(current_path) != current_path:
        current_path = os.path.dirname(current_path)
    
    if os.path.basename(current_path) == 'rl-llm':
        os.chdir(current_path)
        print(f"Changed directory to: {current_path}")
    else:
        print("Not in rl-llm repository structure")
else:
    print("Already in rl-llm root directory")

print(f"Current directory: {os.getcwd()}")

Changed directory to: /home/pavan/rl-llm
Current directory: /home/pavan/rl-llm


In [2]:
from src import EnvManager, sample_episodes
import time
import matplotlib.pyplot as plt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-GoTo-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Open-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-Pickup-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/pavan/rl-llm/.conda/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment BabyAI-PutNext-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "meta-llama/Llama-3.2-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
generation_kwargs = {
    "max_new_tokens": 20,
    "do_sample": True,
    "top_k": 10,
    "top_p": 0.95,
    "temperature": 0.8,
    "pad_token_id": tokenizer.pad_token_id,
}
env_ids = ["BabyAI-GoTo-v0"]
num_envs = 1
env_managers = [
    EnvManager(
        env_ids, 
        invalid_action_penalty=-2,
        consecutive_invalid_actions_allowed=5,
    )
    for i in range(num_envs)
]
print("Sampling batch...")
start_time = time.time()
stats = sample_episodes(
    envs=env_managers,
    tokenizer=tokenizer,
    model=model,
    generation_kwargs=generation_kwargs,
    device=device,
    number_of_episodes=1,
    context_window=5,
    reasoning_flag=False,
)
elapsed_time = time.time() - start_time
print("Summary of stats:")
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Success rate: {stats['all_success_rate']:.2f}")
print(f"Average reward: {stats['all_avg_reward']:.2f}")
print(f"Average episode length: {stats['all_avg_episode_length']:.2f}")
print(f"Average number of invalid actions: {stats['all_avg_invalid_actions']:.2f}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Sampling batch...
Summary of stats:
Elapsed time: 14.33 seconds
Success rate: 0.00
Average reward: 0.00
Average episode length: 26.00
Average number of invalid actions: 5.00
